In [3]:
import pandas as pd
import numpy as np
from collections import defaultdict

class ARule:
    def __init__(self,s,l,sn=0,ln=0):
        self.s = s
        self.l = l
        self.sn = sn
        self.ln = ln

class Apriori:
    def __init__(self,minSuppCount = 2,minConfidence=0.5):
        self.df = None
        self.file = None
        self.basicItem = None
        self.rawSet = None
        self.minSuppCount = minSuppCount
        self.minConfidence = minConfidence * 100
        
    def read_csv(self,file):
        #https://stackoverflow.com/questions/27020216/import-csv-with-different-number-of-columns-per-row-using-pandas
        df = pd.read_fwf(file, header=None)
        df = df[0].str.split(',', expand=True)
        self.df = df
#         print(df)
        #https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_numpy.html
    
    def resolveDuplicated(self):
        rawSet = self.df.to_numpy()
        for nRow,row in enumerate(rawSet):
            counter = 1
            for nCol,col in enumerate(row):
        #         print(row)
        #         print(len(row))
        #         print(col)
                if col == None:
                    continue
                for restN in range(nCol+1,len(row)):
        #             print(restN)
                    if(col == row[restN]):
                        row[restN] = str(col) + str(counter)
    #                     print(f'rowID:{nRow} {col}, {row[restN]}{counter}')
                        counter = counter + 1
        #         print(f'{col},{nRow},{nCol}')
        #     print('=== End each row ===')
        self.basicItem = np.unique(rawSet[rawSet != np.array(None)])
        self.rawSet = rawSet.astype('object')
        return self.basicItem
    
    def getNextItemset(self,itemset, k):
        basicItem = itemset
        newBasicItem = []
        for n,item in enumerate(basicItem):
            for j in range(n+1,len(basicItem)):
                item1 = item
                item2 = basicItem[j]
                itemSet = np.union1d(item1,item2)
                if(len(itemSet) == k):
                    newBasicItem.append(np.array(itemSet))
            uniqueBasicItem = np.unique(np.array(newBasicItem),axis=0)
            uniqueBasicItem.astype(object)
        return uniqueBasicItem
    
    def calculateMinSuppCount(self,basicItem):
        #https://stackoverflow.com/questions/56419519/selecting-rows-of-numpy-array-that-contains-all-the-given-values
        item = basicItem
        
        if(type(item) == np.str):
            k = 1
        elif(type(item) == np.ndarray):
            k = len(item)
        else:
#             print('itemset length error! line 66')
            k = len(item)
            
#         print(item)
        mask = np.isin(self.rawSet,item)
#         print(mask.sum(axis=1))
        vec_mask = np.isin(mask.sum(axis=1), [k])
        ids = np.where(vec_mask)
#         existence = ap.rawSet[ids]
        minSuppCount = len(ids[0])
        return minSuppCount
    
    def getKey(self,basicItem):
        if(type(basicItem) == np.str):
            return basicItem
        elif(type(basicItem) == np.str_):
            return basicItem
        elif(type(basicItem) == np.ndarray):
            return basicItem.tobytes()
        else:
            print('key error! line 83')
    
    def getMinSupportItemset(self,basicItemSet):
        suppCountDict = {}
        for basicItem in basicItemSet:
            suppCountDict[self.getKey(basicItem)] = self.calculateMinSuppCount(basicItem)
        return suppCountDict
    
    def filterItemsetwithCount(self,itemset,suppCountDict):
        minSuppCount = self.minSuppCount
        newItemset = []
        counter = 1
        for item in itemset:
            count = suppCountDict[ap.getKey(item)]
            if(count >= minSuppCount):
                newItemset.append(item)
        return np.array(newItemset)
    
    def displayItemsetCount(self,itemset,suppCountDict):
        minSuppCount = self.minSuppCount
        try:
            k = itemset.shape[1]
        except:
            k = 1
        counter = 1
#         print ('{:=^40}'.format(f' {k}-ItemSet '))
        for item in itemset:
#             print('{:^40}'.format(f'{counter}: {item}: {suppCountDict[self.getKey(item)]}'))
            print(f'{counter}: {item}: {suppCountDict[self.getKey(item)]}')        
            counter = counter + 1
            

#         print ('{}'.format(f' Item that sastify min support count ({minSuppCount}) '))
        
#         counter = 1
#         for item in itemset:
#             count = suppCountDict[self.getKey(item)]
#             if(count >= minSuppCount):
#                 print(f'{counter}: {item}: {suppCountDict[self.getKey(item)]}')
#                 counter = counter+1
#         print ('{:=^40}'.format(f' End of {k}-ItemSet '))

    def displayStartDivider(self,counter):
        print('\n')
        print ('{:-^40}'.format(f' {counter}-itemset '))
        print('\n')
#         print ('{:-^40}'.format(f'RowID: Itemset: Count'))
    
    def displayMinSuppCountMsg(self):
        print('\n')
        print ('{:*^40}'.format(f' Item that sastify min support count ({self.minSuppCount}) '))
        print('\n')
        
    def displayFrequentPattern(self):
        print ('{:.^40}'.format(f' Frequent Patterns '))
        for fp in self.frequentPattern:
            itemset = fp[0]
            itemdict = fp[1]
            self.displayItemsetCount(itemset,itemdict)
            
    def associationRule(self):
        threeItemsetAbove = []
        for fp in self.frequentPattern[2:]:
            threeItemsetAbove.append(fp[0])
            rulesSet = self.generateRules(threeItemsetAbove)
        return threeItemsetAbove,rulesSet

    # https://www.youtube.com/watch?v=9oPNGofa1pI&t=32s
    def powerset(self,originalSet):
        subSets = []
        numberOfCombinations = 2 ** len(originalSet) - 1
        for combinationIndex in range(1,numberOfCombinations):
            subSet = []
            for setElementIndex in range(0, len(originalSet)):
                if combinationIndex & 1 << setElementIndex:
                    subSet.append(originalSet[setElementIndex])
            subSets.append(subSet)
        return subSets    

    def generateRules(self,threeItemsetAbove):
        rulesSet = defaultdict(list)
        for itemsets in threeItemsetAbove:
            for itemset in itemsets:
                subsets = self.powerset(itemset)
                for n,subset1 in enumerate(subsets):
                    for subset2 in subsets:
                        sn = 0
                        ln = 0
                        mask = np.isin(subset1,subset2)
                        numberOfConflict = mask.sum()
                        if(numberOfConflict == 0):
                            if(len(subset1) != len(subset2)):
                                sn = self.calculateMinSuppCount(subset1)
                                ln = self.calculateMinSuppCount(subset2+subset1)
                                percentage = (ln/sn) * 100
                                if (percentage >= self.minConfidence):
                                    rulesSet[self.getKey(itemset)].append(ARule(subset1,subset2,sn,ln))
            return rulesSet

    def displayRules(self,threeItemsetAbove,rulesSet):
        for itemsets in threeItemsetAbove:
            for itemset in itemsets:
                rules = rulesSet[self.getKey(itemset)]
                print(itemset)
                for rule in rules:
                    percentage = (rule.ln/rule.sn) * 100
                    print(f'{rule.s} => {rule.l} confidence={rule.ln}/{rule.sn} = {percentage}')    

    def process(self,file):
        print ('{:=^40}'.format(f' Start of apriori '))
        self.file = file
        self.read_csv(file)
        self.resolveDuplicated()
        frequentPattern = []

        basicItem = self.basicItem
        itemDict = self.getMinSupportItemset(basicItem)
        self.itemDict = itemDict
        # print ('{:=^40}'.format(f' 1-itemset '))
        self.displayStartDivider(1)
        self.displayItemsetCount(basicItem,itemDict)
        ItemFiltered = self.filterItemsetwithCount(basicItem,itemDict)
        # print ('{}'.format(f' Item that sastify min support count ({ap.minSuppCount}) '))
        self.displayMinSuppCountMsg()
        self.displayItemsetCount(ItemFiltered,itemDict)
        frequentPattern.append([ItemFiltered,itemDict])
        counter = 2
        while ItemFiltered.size > 2:
            self.displayStartDivider(counter)
        #     print ('{:=^40}'.format(f' {counter}-itemset '))
            nextItemset = ap.getNextItemset(ItemFiltered,counter)
            nextItemsetCount = ap.getMinSupportItemset(nextItemset)
            self.displayItemsetCount(nextItemset,nextItemsetCount)
            ItemFiltered = self.filterItemsetwithCount(nextItemset,nextItemsetCount)
            if(ItemFiltered.size != 0):
                self.displayMinSuppCountMsg()
            self.displayItemsetCount(ItemFiltered,nextItemsetCount)
            counter = counter+1
            if(ItemFiltered.size != 0):
                frequentPattern.append([ItemFiltered,nextItemsetCount])
        self.frequentPattern = frequentPattern
        self.displayFrequentPattern()
        print('')
        print ('{:-^40}'.format(f' Strong Rule '))
        print('')
        try:
            rulesSet = self.associationRule()
            threeItemsetAbove, rulesSet = self.associationRule()
            self.displayRules(threeItemsetAbove,rulesSet)
        except:
            print('\nNo strong rule\n')
        # print ('{:^40}'.format(f' None '))
        print('\n')
        print ('{:=^40}'.format(f' End of apriori '))

In [4]:
file = 'item.csv'
ap = Apriori(2,0.5)
ap.process(file)

=========== Start of apriori ===========


-------------- 1-itemset ---------------


1: I1: 6
2: I2: 7
3: I3: 6
4: I4: 2
5: I5: 2


 Item that sastify min support count (2) 


1: I1: 6
2: I2: 7
3: I3: 6
4: I4: 2
5: I5: 2


-------------- 2-itemset ---------------


1: ['I1' 'I2']: 4
2: ['I1' 'I3']: 4
3: ['I1' 'I4']: 1
4: ['I1' 'I5']: 2
5: ['I2' 'I3']: 4
6: ['I2' 'I4']: 2
7: ['I2' 'I5']: 2
8: ['I3' 'I4']: 0
9: ['I3' 'I5']: 1
10: ['I4' 'I5']: 0


 Item that sastify min support count (2) 


1: ['I1' 'I2']: 4
2: ['I1' 'I3']: 4
3: ['I1' 'I5']: 2
4: ['I2' 'I3']: 4
5: ['I2' 'I4']: 2
6: ['I2' 'I5']: 2


-------------- 3-itemset ---------------


1: ['I1' 'I2' 'I3']: 2
2: ['I1' 'I2' 'I4']: 1
3: ['I1' 'I2' 'I5']: 2
4: ['I1' 'I3' 'I5']: 1
5: ['I2' 'I3' 'I4']: 0
6: ['I2' 'I3' 'I5']: 1
7: ['I2' 'I4' 'I5']: 0


 Item that sastify min support count (2) 


1: ['I1' 'I2' 'I3']: 2
2: ['I1' 'I2' 'I5']: 2


-------------- 4-itemset ---------------


1: ['I1' 'I2' 'I3' 'I5']: 1
.......... Frequent Pattern

In [514]:
for fp in ap.frequentPattern:
    itemset = fp[0]
    itemdict = fp[1]
    print(itemset)

['I1' 'I2' 'I3' 'I4' 'I5']
[['I1' 'I2']
 ['I1' 'I3']
 ['I1' 'I5']
 ['I2' 'I3']
 ['I2' 'I4']
 ['I2' 'I5']]
[['I1' 'I2' 'I3']
 ['I1' 'I2' 'I5']]


In [528]:
def associationRule(self):
    threeItemsetAbove = []
    for fp in self.frequentPattern[2:]:
        threeItemsetAbove.append(fp[0])
        
    self.generateRules(threeItemsetAbove)
    
# https://www.youtube.com/watch?v=9oPNGofa1pI&t=32s
def powerset(self,originalSet):
    subSets = []
    numberOfCombinations = 2 ** len(originalSet) - 1
    for combinationIndex in range(1,numberOfCombinations):
        subSet = []
        for setElementIndex in range(0, len(originalSet)):
            if combinationIndex & 1 << setElementIndex:
                subSet.append(originalSet[setElementIndex])
        subSets.append(subSet)
    return subSets

from collections import defaultdict
rulesSet = defaultdict(list)

def generateRules(self,threeItemsetAbove):
    for itemsets in threeItemsetAbove:
        for itemset in itemsets:
            subsets = powerset(itemset)
            for n,subset1 in enumerate(subsets):
                for subset2 in subsets:
                    sn = 0
                    ln = 0
                    mask = np.isin(subset1,subset2)
                    numberOfConflict = mask.sum()
                    if(numberOfConflict == 0):
                        if(len(subset1) != len(subset2)):
                            sn = ap.calculateMinSuppCount(subset1)
                            ln = ap.calculateMinSuppCount(subset2+subset1)
                            percentage = (ln/sn) * 100
                            rulesSet[ap.getKey(itemset)].append(ARule(subset1,subset2,sn,ln))
        return rulesSet

def displayRules(self,threeItemsetAbove):
    for itemsets in threeItemsetAbove:
        for itemset in itemsets:
            rules = rulesSet[ap.getKey(itemset)]
            print(itemset)
            for rule in rules:
                percentage = (rule.ln/rule.sn) * 100
                print(f'{rule.s} => {rule.l} confidence={rule.ln}/{rule.sn} = {percentage}')

In [515]:
threeItemsetAbove = []
for fp in ap.frequentPattern[2:]:
    threeItemsetAbove.append(fp[0])

In [516]:
print(threeItemsetAbove)

[array([['I1', 'I2', 'I3'],
       ['I1', 'I2', 'I5']], dtype='<U2')]


In [517]:
for itemsets in threeItemsetAbove:
    for itemset in itemsets:
        print(powerset(itemset))

[['I1'], ['I2'], ['I1', 'I2'], ['I3'], ['I1', 'I3'], ['I2', 'I3']]
[['I1'], ['I2'], ['I1', 'I2'], ['I5'], ['I1', 'I5'], ['I2', 'I5']]


In [527]:
threeItemsetAbove = []
for fp in ap.frequentPattern[2:]:
    threeItemsetAbove.append(fp[0])
    
# https://www.youtube.com/watch?v=9oPNGofa1pI&t=32s
def powerset(originalSet):
    subSets = []
    numberOfCombinations = 2 ** len(originalSet) - 1
    for combinationIndex in range(1,numberOfCombinations):
        subSet = []
        for setElementIndex in range(0, len(originalSet)):
            if combinationIndex & 1 << setElementIndex:
                subSet.append(originalSet[setElementIndex])
        subSets.append(subSet)
    return subSets

from collections import defaultdict
rulesSet = defaultdict(list)
# rulesSet = {}

for itemsets in threeItemsetAbove:
    for itemset in itemsets:
#         print('=======')
#         print(itemset)
        subsets = powerset(itemset)
        for n,subset1 in enumerate(subsets):
            for subset2 in subsets:
                sn = 0
                ln = 0
                mask = np.isin(subset1,subset2)
                numberOfConflict = mask.sum()
                if(numberOfConflict == 0):
                    if(len(subset1) != len(subset2)):
                        sn = ap.calculateMinSuppCount(subset1)
                        ln = ap.calculateMinSuppCount(subset2+subset1)
                        percentage = (ln/sn) * 100
        #                 print(f'{subset1} => {subset2} confidence={ln}/{sn} = {percentage}, {subset1+subset2}')
                        rulesSet[ap.getKey(itemset)].append(ARule(subset1,subset2,sn,ln))
        #                 rule.append([subset1,subset2]  
# print('==')
# for i in rules:
#     percentage = (i.ln/i.sn) * 100
#     print(f'{i.s} => {i.l} confidence={i.ln}/{i.sn} = {percentage}')

# for key in rulesSet:
#     rules = rulesSet[key]
#     print(key)
#     for rule in rules:
#         percentage = (rule.ln/rule.sn) * 100
#         print(f'{rule.s} => {rule.l} confidence={rule.ln}/{rule.sn} = {percentage}')

for itemsets in threeItemsetAbove:
    for itemset in itemsets:
        rules = rulesSet[ap.getKey(itemset)]
        print(itemset)
        for rule in rules:
            percentage = (rule.ln/rule.sn) * 100
            print(f'{rule.s} => {rule.l} confidence={rule.ln}/{rule.sn} = {percentage}')

['Beer' 'Bread' 'Cola']
['Beer'] => ['Bread', 'Cola'] confidence=2/7 = 28.57142857142857
['Bread'] => ['Beer', 'Cola'] confidence=2/9 = 22.22222222222222
['Beer', 'Bread'] => ['Cola'] confidence=2/6 = 33.33333333333333
['Cola'] => ['Beer', 'Bread'] confidence=2/4 = 50.0
['Beer', 'Cola'] => ['Bread'] confidence=2/3 = 66.66666666666666
['Bread', 'Cola'] => ['Beer'] confidence=2/3 = 66.66666666666666
['Beer' 'Bread' 'Diapers']
['Beer'] => ['Bread', 'Diapers'] confidence=4/7 = 57.14285714285714
['Bread'] => ['Beer', 'Diapers'] confidence=4/9 = 44.44444444444444
['Beer', 'Bread'] => ['Diapers'] confidence=4/6 = 66.66666666666666
['Diapers'] => ['Beer', 'Bread'] confidence=4/6 = 66.66666666666666
['Beer', 'Diapers'] => ['Bread'] confidence=4/5 = 80.0
['Bread', 'Diapers'] => ['Beer'] confidence=4/5 = 80.0
['Beer' 'Bread' 'Milk']
['Beer'] => ['Bread', 'Milk'] confidence=4/7 = 57.14285714285714
['Bread'] => ['Beer', 'Milk'] confidence=4/9 = 44.44444444444444
['Beer', 'Bread'] => ['Milk'] confid

In [167]:
ap.basicItemset(itemset)

AttributeError: 'Apriori' object has no attribute 'basicItemset'

In [128]:
itemset = ap.frequentPattern[-1][0][1]
print(itemset)

['I1' 'I2' 'I5']


In [224]:
a = ['I1','I2','I3']
b = ['I3','I1']
c = ['I3']

In [237]:
mask = np.isin(a,b)
vec_mask = mask.sum()
print(mask)
print(vec_mask)

[ True False  True]
2


In [ ]:
mask = np.isin(a,b)
#         print(mask.sum(axis=1))
vec_mask = np.isin(mask.sum(axis=1), [k])
ids = np.where(vec_mask)
#         existence = ap.rawSet[ids]
minSuppCount = len(ids[0])
return minSuppCount

In [213]:
print(np.any(b,c))

TypeError: cannot perform reduce with flexible type

In [103]:
# https://www.youtube.com/watch?v=9oPNGofa1pI&t=32s
def powerset(originalSet):
    subSets = []
    numberOfCombinations = 2 ** len(originalSet) - 1
    for combinationIndex in range(1,numberOfCombinations):
        subSet = []
        for setElementIndex in range(0, len(originalSet)):
            if combinationIndex & 1 << setElementIndex:
                subSet.append(originalSet[setElementIndex])
        subSets.append(subSet)
    return subSets

In [129]:
powerset([1,2,3,4])

[[1],
 [2],
 [1, 2],
 [3],
 [1, 3],
 [2, 3],
 [1, 2, 3],
 [4],
 [1, 4],
 [2, 4],
 [1, 2, 4],
 [3, 4],
 [1, 3, 4],
 [2, 3, 4]]

In [183]:
# https://github.com/chonyy/apriori_python/blob/70c45cad9ad08cb78e57787478568d49c18c71df/apriori_python/utils.py#L77
rules = []
subsets = powerset(itemset)
# print(subsets)
for subset in subsets:
    print(f'{subset}: {ap.calculateMinSuppCount(subset)}')

['I1']: 6
['I2']: 7
['I1', 'I2']: 4
['I5']: 2
['I1', 'I5']: 2
['I2', 'I5']: 2


In [274]:
gfg = np.intersect1d(np.array(subsets,dtype=object), np.array(subsets,dtype=object)) 
    
for i in gfg:
    print(i)

['I1']
['I1', 'I2']
['I1', 'I5']
['I2']
['I2', 'I5']
['I5']


In [326]:
class ARule:
    def __init__(self,s,l,sn=0,ln=0):
        self.s = s
        self.l = l
        self.sn = sn
        self.ln = ln

In [311]:
ap.calculateMinSuppCount(['I2','I5'])

2

In [309]:
print(ap.rawSet)

[['I1' 'I2' 'I5' None]
 ['I2' 'I4' None None]
 ['I2' 'I3' None None]
 ['I1' 'I2' 'I4' None]
 ['I1' 'I3' None None]
 ['I2' 'I3' None None]
 ['I1' 'I3' None None]
 ['I1' 'I2' 'I3' 'I5']
 ['I1' 'I2' 'I3' None]]


In [418]:
rule = []
for n,subset1 in enumerate(subsets):
    for subset2 in subsets:
        sn = 0
        ln = 0
        mask = np.isin(subset1,subset2)
        numberOfConflict = mask.sum()
        if(numberOfConflict == 0):
            if(len(subset1) != len(subset2)):
                sn = ap.calculateMinSuppCount(subset1)
                ln = ap.calculateMinSuppCount(subset2+subset1)
                percentage = (ln/sn) * 100
#                 print(f'{subset1} => {subset2} confidence={ln}/{sn} = {percentage}, {subset1+subset2}')
                rule.append(ARule(subset1,subset2,sn,ln))
#                 rule.append([subset1,subset2]  
# print('==')
for i in rule:
    percentage = (i.ln/i.sn) * 100
    print(f'{i.s} => {i.l} confidence={i.ln}/{i.sn} = {percentage}')

['I1'] => ['I2', 'I5'] confidence=2/6 = 33.33333333333333
['I2'] => ['I1', 'I5'] confidence=2/7 = 28.57142857142857
['I1', 'I2'] => ['I5'] confidence=2/4 = 50.0
['I5'] => ['I1', 'I2'] confidence=2/2 = 100.0
['I1', 'I5'] => ['I2'] confidence=2/2 = 100.0
['I2', 'I5'] => ['I1'] confidence=2/2 = 100.0


In [369]:
itemset = ap.frequentPattern[-1][0]
print(itemset)

[['I1' 'I2' 'I3']
 ['I1' 'I2' 'I5']]


In [421]:
from collections import defaultdict
rulesSet = defaultdict(list)
# rulesSet = {}
for itemset in ap.frequentPattern[-1][0]:
    print(itemset)
    subsets = powerset(itemset)
    for n,subset1 in enumerate(subsets):
        for subset2 in subsets:
            sn = 0
            ln = 0
            mask = np.isin(subset1,subset2)
            numberOfConflict = mask.sum()
            if(numberOfConflict == 0):
                if(len(subset1) != len(subset2)):
                    sn = ap.calculateMinSuppCount(subset1)
                    ln = ap.calculateMinSuppCount(subset2+subset1)
                    percentage = (ln/sn) * 100
    #                 print(f'{subset1} => {subset2} confidence={ln}/{sn} = {percentage}, {subset1+subset2}')
                    rulesSet[ap.getKey(itemset)].append(ARule(subset1,subset2,sn,ln))
    #                 rule.append([subset1,subset2]  
# print('==')
# for i in rules:
#     percentage = (i.ln/i.sn) * 100
#     print(f'{i.s} => {i.l} confidence={i.ln}/{i.sn} = {percentage}')

['I1' 'I2' 'I3']
['I1' 'I2' 'I5']


In [429]:
for key in rulesSet:
    rules = rulesSet[key]
    print(key)
    for rule in rules:
        percentage = (rule.ln/rule.sn) * 100
        print(f'{rule.s} => {rule.l} confidence={rule.ln}/{rule.sn} = {percentage}')

b'I\x00\x00\x001\x00\x00\x00I\x00\x00\x002\x00\x00\x00I\x00\x00\x003\x00\x00\x00'
['I1'] => ['I2', 'I3'] confidence=2/6 = 33.33333333333333
['I2'] => ['I1', 'I3'] confidence=2/7 = 28.57142857142857
['I1', 'I2'] => ['I3'] confidence=2/4 = 50.0
['I3'] => ['I1', 'I2'] confidence=2/6 = 33.33333333333333
['I1', 'I3'] => ['I2'] confidence=2/4 = 50.0
['I2', 'I3'] => ['I1'] confidence=2/4 = 50.0
b'I\x00\x00\x001\x00\x00\x00I\x00\x00\x002\x00\x00\x00I\x00\x00\x005\x00\x00\x00'
['I1'] => ['I2', 'I5'] confidence=2/6 = 33.33333333333333
['I2'] => ['I1', 'I5'] confidence=2/7 = 28.57142857142857
['I1', 'I2'] => ['I5'] confidence=2/4 = 50.0
['I5'] => ['I1', 'I2'] confidence=2/2 = 100.0
['I1', 'I5'] => ['I2'] confidence=2/2 = 100.0
['I2', 'I5'] => ['I1'] confidence=2/2 = 100.0


In [411]:
for itemset in rulesSet:
    rule = rulesSet[itemset]
    print(itemset)
    for r in rule:
        print(r)

In [395]:

print(itemset.tobytes())

b'I\x00\x00\x001\x00\x00\x00I\x00\x00\x002\x00\x00\x00I\x00\x00\x003\x00\x00\x00'


In [353]:
print(rule[-1].l+(rule[-1].s))

['I1', 'I2', 'I5']


In [350]:
test = rule[-1].l
ap.calculateMinSuppCount(test)

6

In [664]:
itemset[0]

array(['I1', 'I2', 'I3'], dtype='<U2')

In [318]:
ap.calculateMinSuppCount(rule[-1].s)

2

In [324]:
print(ap.calculateMinSuppCount(subsets[-1]))
print(subsets[-1])

2
['I2', 'I5']


In [312]:
for i in rule:
    print(i)

In [155]:
a_p1 = a_p2 = subsets
from sklearn.utils.extmath import cartesian
print(cartesian([a_p1, a_p2]))

ModuleNotFoundError: No module named 'sklearn'

In [230]:
# https://stackoverflow.com/questions/44409084/how-to-zip-two-1d-numpy-array-to-2d-numpy-array
# https://www.mathsisfun.com/sets/symbols.html
z = np.array(list(zip(subsets, subsets)))
print(z)

[[list(['I1']) list(['I1'])]
 [list(['I2']) list(['I2'])]
 [list(['I1', 'I2']) list(['I1', 'I2'])]
 [list(['I5']) list(['I5'])]
 [list(['I1', 'I5']) list(['I1', 'I5'])]
 [list(['I2', 'I5']) list(['I2', 'I5'])]]


<ipython-input-230-9defcb399a82>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  z = np.array(list(zip(subsets, subsets)))


In [232]:
# https://stackoverflow.com/questions/11144513/cartesian-product-of-x-and-y-array-points-into-single-array-of-2d-points
x = y = subsets
z= np.transpose([np.tile(x, len(y)), np.repeat(y, len(x))])
print(z)

[[list(['I1']) list(['I1'])]
 [list(['I2']) list(['I1'])]
 [list(['I1', 'I2']) list(['I1'])]
 [list(['I5']) list(['I1'])]
 [list(['I1', 'I5']) list(['I1'])]
 [list(['I2', 'I5']) list(['I1'])]
 [list(['I1']) list(['I2'])]
 [list(['I2']) list(['I2'])]
 [list(['I1', 'I2']) list(['I2'])]
 [list(['I5']) list(['I2'])]
 [list(['I1', 'I5']) list(['I2'])]
 [list(['I2', 'I5']) list(['I2'])]
 [list(['I1']) list(['I1', 'I2'])]
 [list(['I2']) list(['I1', 'I2'])]
 [list(['I1', 'I2']) list(['I1', 'I2'])]
 [list(['I5']) list(['I1', 'I2'])]
 [list(['I1', 'I5']) list(['I1', 'I2'])]
 [list(['I2', 'I5']) list(['I1', 'I2'])]
 [list(['I1']) list(['I5'])]
 [list(['I2']) list(['I5'])]
 [list(['I1', 'I2']) list(['I5'])]
 [list(['I5']) list(['I5'])]
 [list(['I1', 'I5']) list(['I5'])]
 [list(['I2', 'I5']) list(['I5'])]
 [list(['I1']) list(['I1', 'I5'])]
 [list(['I2']) list(['I1', 'I5'])]
 [list(['I1', 'I2']) list(['I1', 'I5'])]
 [list(['I5']) list(['I1', 'I5'])]
 [list(['I1', 'I5']) list(['I1', 'I5'])]
 [list(['

In [252]:
v = []
for i in z:
    s = i[0]
    l = i[1]
    mask = np.isin(s,l)
    mask2 = np.isin(l,s)
    vec_mask = mask.sum()
    vec_mask2 = mask2.sum()
    if(vec_mask == 0 and vec_mask2 == 0):
        v.append([s,l])
v = np.array(v,dtype=object)

In [253]:
for i in v:
    print(i)

[list(['I2']) list(['I1'])]
[list(['I5']) list(['I1'])]
[list(['I2', 'I5']) list(['I1'])]
[list(['I1']) list(['I2'])]
[list(['I5']) list(['I2'])]
[list(['I1', 'I5']) list(['I2'])]
[list(['I5']) list(['I1', 'I2'])]
[list(['I1']) list(['I5'])]
[list(['I2']) list(['I5'])]
[list(['I1', 'I2']) list(['I5'])]
[list(['I2']) list(['I1', 'I5'])]
[list(['I1']) list(['I2', 'I5'])]


In [198]:
import itertools
for element in itertools.product(subsets):
    print(element)

(['I1'],)
(['I2'],)
(['I1', 'I2'],)
(['I5'],)
(['I1', 'I5'],)
(['I2', 'I5'],)


In [195]:
print(z[3])

[list(['I5']) list(['I1'])]


In [145]:
for x,y in z:
    print(x,y)

['I1'] ['I1']
['I2'] ['I2']
['I1', 'I2'] ['I1', 'I2']
['I5'] ['I5']
['I1', 'I5'] ['I1', 'I5']
['I2', 'I5'] ['I2', 'I5']


In [132]:
print(itemset)

['I1' 'I2' 'I5']


In [664]:
itemset[0]

array(['I1', 'I2', 'I3'], dtype='<U2')

In [80]:
for fp in ap.frequentPattern:
    itemset = fp[0]
    itemdict = fp[1]
    ap.displayItemsetCount(itemset,itemdict)

1: I1: 6
2: I2: 7
3: I3: 6
4: I4: 2
5: I5: 2
1: ['I1' 'I2']: 4
2: ['I1' 'I3']: 4
3: ['I1' 'I5']: 2
4: ['I2' 'I3']: 4
5: ['I2' 'I4']: 2
6: ['I2' 'I5']: 2
1: ['I1' 'I2' 'I3']: 2
2: ['I1' 'I2' 'I5']: 2


In [4]:
fp = ap.frequentPattern[-1]
itemset = fp[0]
itemdict = fp[1]
print(itemset)

[['I1' 'I2' 'I3']
 ['I1' 'I2' 'I5']]


In [682]:
base = []

In [5]:
def classical_recursive_two(elems):
    """ Given a list of elements return a generator 
    that will generate all the subsets"""
    if  len(elems) <= 1:
        yield elems
        yield []
    else:
        for item in classical_recursive_two(elems[1:]):
            yield [elems[0]] + item
            yield item

In [36]:
itemset

array([['I1', 'I2', 'I3'],
       ['I1', 'I2', 'I5']], dtype='<U2')

In [46]:
#https://codereview.stackexchange.com/questions/147633/get-subsets-of-a-set-given-as-a-list
#https://www.youtube.com/watch?v=9oPNGofa1pI
def subsets(s):
    """
    :type s: list[]
    """
    sets = []
    for i in range(2**len(s)):
        subset = []
        for j in range(len(s)):
            if i & (1 << j) > 0:
                subset.append(s[j])
        sets.append(subset)
    return sets

In [47]:
print(subsets(itemset[0]))

[[], ['I1'], ['I2'], ['I1', 'I2'], ['I3'], ['I1', 'I3'], ['I2', 'I3'], ['I1', 'I2', 'I3']]


In [75]:
#https://codereview.stackexchange.com/questions/147633/get-subsets-of-a-set-given-as-a-list
#https://www.youtube.com/watch?v=9oPNGofa1pI
#https://www.mathsisfun.com/sets/power-set.html
def subsetssss(s):
    """
    :type s: list[]
    """
    sets = []
    for i in range(2**len(s)):
        subset = []
        for j in range(len(s)):
            if i & (1 << j) > 0:
                print(s[j])
                subset.append(s[j])
        sets.append(subset)
    return sets

In [78]:
subsetssss(['b','c','l','s'])

b
c
b
c
l
b
l
c
l
b
c
l
s
b
s
c
s
b
c
s
l
s
b
l
s
c
l
s
b
c
l
s


[[],
 ['b'],
 ['c'],
 ['b', 'c'],
 ['l'],
 ['b', 'l'],
 ['c', 'l'],
 ['b', 'c', 'l'],
 ['s'],
 ['b', 's'],
 ['c', 's'],
 ['b', 'c', 's'],
 ['l', 's'],
 ['b', 'l', 's'],
 ['c', 'l', 's'],
 ['b', 'c', 'l', 's']]

In [43]:
np.array(subsets(itemset[0]))

<ipython-input-43-d4f43103af67>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.array(subsets(itemset[0]))


array([list([]), list(['I1']), list(['I2']), list(['I1', 'I2']),
       list(['I3']), list(['I1', 'I3']), list(['I2', 'I3']),
       list(['I1', 'I2', 'I3'])], dtype=object)

In [693]:
#https://www.geeksforgeeks.org/python-print-sublists-list/
powerset = itemset[0]
subset = []
for i in range(len(powerset)):
    orig = subset[:]
    new=powerset[i]
    print(new)
    for j in range(len(subset)):
        print(subset[j])
        subset[j].append(new)
    subset = orig + subset
        

I1
I2
I3


In [703]:
#https://stackoverflow.com/questions/374626/how-can-i-find-all-the-subsets-of-a-set-with-exactly-n-elements
import itertools
def findsubsets(S,m):
    return set(itertools.combinations(S, m))


In [704]:
findsubsets(itemset[0],2)

{('I1', 'I2'), ('I1', 'I3'), ('I2', 'I3')}

In [ ]:
#https://www.geeksforgeeks.org/python-program-to-get-all-subsets-of-given-size-of-a-set/

In [694]:
#https://stackoverflow.com/questions/18826571/python-powerset-of-a-given-set-with-generators
#https://stackoverflow.com/questions/26332412/python-recursive-function-to-display-all-subsets-of-given-set

In [691]:
print(subset)

[]


In [679]:
print(lists)

[[]]


In [674]:
#https://www.technomancy.org/python/powerset-generator-python/

def powerset(seq):
    """
    Returns all the subsets of this set. This is a generator.
    """
    if len(seq) <= 1:
        yield seq
        yield []
    else:
        for item in powerset(seq[1:]):
            yield [seq[0]]+item
            yield item

In [675]:
powerset(itemset[0])

<generator object powerset at 0x0000025F6E8C09E0>

In [672]:
subset

[array(['I1', 'I2'], dtype='<U2'),
 array(['I1', 'I3'], dtype='<U2'),
 array(['I2', 'I3'], dtype='<U2')]

In [ ]:
for fp in ap.frequentPattern:
    itemset = fp[0]
    itemdict = fp[1]
    print(itemset)
    print(itemdict)

In [379]:
def filterItemsetwithCount(itemset,suppCountDict):
    minSuppCount = 2
    newItemset = []
    counter = 1
    for item in itemset:
        count = suppCountDict[ap.getKey(item)]
        if(count >= minSuppCount):
            newItemset.append(item)
    return np.array(newItemset)

In [387]:
newItem2 = filterItemsetwithCount(basicItem,itemDict)
print(newItem2)
ap.printItemsetCount(newItem2,itemDict)

['Beer' 'Bread' 'Cola' 'Diapers' 'Milk']
1: Beer: 7
2: Bread: 9
3: Cola: 4
4: Diapers: 6
5: Milk: 8


In [533]:
file = 'item.csv'
print ('{:=^40}'.format(f' Start of apriori '))
ap = Apriori(3)
ap.read_csv(file)
ap.resolveDuplicated()
frequentPattern = []

basicItem = ap.basicItem
itemDict = ap.getMinSupportItemset(basicItem)
# print ('{:=^40}'.format(f' 1-itemset '))
ap.displayStartDivider(1)
ap.printItemsetCount(basicItem,itemDict)
ItemFiltered = ap.filterItemsetwithCount(basicItem,itemDict)
# print ('{}'.format(f' Item that sastify min support count ({ap.minSuppCount}) '))
ap.displayMinSuppCountMsg()
ap.printItemsetCount(ItemFiltered,itemDict)
frequentPattern.append(ItemFiltered)
counter = 2
while ItemFiltered.size > 2:
    ap.displayStartDivider(counter)
#     print ('{:=^40}'.format(f' {counter}-itemset '))
    nextItemset = ap.getNextItemset(ItemFiltered,counter)
    nextItemsetCount = ap.getMinSupportItemset(nextItemset)
    ap.printItemsetCount(nextItemset,nextItemsetCount)
    ItemFiltered = ap.filterItemsetwithCount(nextItemset,nextItemsetCount)
    ap.displayMinSuppCountMsg()
    ap.printItemsetCount(ItemFiltered,nextItemsetCount)
    counter = counter+1
    if(ItemFiltered.size != 0):
        frequentPattern.append(ItemFiltered)
# print ('{:^40}'.format(f' None '))
print('\n')
print ('{:=^40}'.format(f' End of apriori '))

=========== Start of apriori ===========


-------------- 1-itemset ---------------


1: I1: 6
2: I2: 7
3: I3: 6
4: I4: 2
5: I5: 2


 Item that sastify min support count (3) 


1: I1: 6
2: I2: 7
3: I3: 6


-------------- 2-itemset ---------------


1: ['I1' 'I2']: 4
2: ['I1' 'I3']: 4
3: ['I2' 'I3']: 4


 Item that sastify min support count (3) 


1: ['I1' 'I2']: 4
2: ['I1' 'I3']: 4
3: ['I2' 'I3']: 4


-------------- 3-itemset ---------------


1: ['I1' 'I2' 'I3']: 2


 Item that sastify min support count (3) 




============ End of apriori ============


In [530]:
frequentPattern[3]

IndexError: list index out of range

In [535]:
for itemset in frequentPattern:
    for item in itemset:
        print(item)

I1
I2
I3
['I1' 'I2']
['I1' 'I3']
['I2' 'I3']


In [425]:
basicItemFiltered.size

5

In [ ]:
nextItemset.size

In [396]:
file = 'food.csv'
ap = Apriori()
ap.read_csv(file)
ap.resolveDuplicated()

basicItem = ap.basicItem
itemDict = ap.getMinSupportItemset(basicItem)
ap.printItemsetCount(basicItem,itemDict)
basicItemFiltered = ap.filterItemsetwithCount(basicItem,itemDict)

item2Set = ap.getNextItemset(basicItemFiltered,2)
item2Dict = ap.getMinSupportItemset(item2Set)
ap.printItemsetCount(item2Set,item2Dict)
item2SetFiltered = ap.filterItemsetwithCount(item2Set,item2Dict)

item3Set = ap.getNextItemset(item2SetFiltered,3)
item3Dict = ap.getMinSupportItemset(item3Set)
ap.printItemsetCount(item3Set,item3Dict)

1: Beer: 7
2: Bread: 9
3: Cola: 4
4: Diapers: 6
5: Diapers1: 1
6: Eggs: 1
7: Milk: 8
1: ['Beer' 'Bread']: 6
2: ['Beer' 'Cola']: 3
3: ['Beer' 'Diapers']: 5
4: ['Beer' 'Milk']: 5
5: ['Bread' 'Cola']: 3
6: ['Bread' 'Diapers']: 5
7: ['Bread' 'Milk']: 7
8: ['Cola' 'Diapers']: 2
9: ['Cola' 'Milk']: 4
10: ['Diapers' 'Milk']: 4
1: ['Beer' 'Bread' 'Cola']: 2
2: ['Beer' 'Bread' 'Diapers']: 4
3: ['Beer' 'Bread' 'Milk']: 4
4: ['Beer' 'Cola' 'Diapers']: 1
5: ['Beer' 'Cola' 'Milk']: 3
6: ['Beer' 'Diapers' 'Milk']: 3
7: ['Bread' 'Cola' 'Diapers']: 1
8: ['Bread' 'Cola' 'Milk']: 3
9: ['Bread' 'Diapers' 'Milk']: 3
10: ['Cola' 'Diapers' 'Milk']: 2


In [274]:
basicItemDict = ap.getMinSupportItemset(basicItem)

In [265]:
print(basicItemDict)

{'Beer': 0, 'Bread': 0, 'Cola': 0, 'Diapers': 0, 'Diapers1': 0, 'Eggs': 0, 'Milk': 0}


In [255]:
type(basicItem[0]) == np.str

True

In [257]:
type(item2Set[0]) == np.ndarray

True

In [236]:
print(itemDict)

{b'I\x00\x00\x001\x00\x00\x00I\x00\x00\x002\x00\x00\x00': 4, b'I\x00\x00\x001\x00\x00\x00I\x00\x00\x003\x00\x00\x00': 4, b'I\x00\x00\x001\x00\x00\x00I\x00\x00\x004\x00\x00\x00': 1, b'I\x00\x00\x001\x00\x00\x00I\x00\x00\x005\x00\x00\x00': 2, b'I\x00\x00\x002\x00\x00\x00I\x00\x00\x003\x00\x00\x00': 4, b'I\x00\x00\x002\x00\x00\x00I\x00\x00\x004\x00\x00\x00': 2, b'I\x00\x00\x002\x00\x00\x00I\x00\x00\x005\x00\x00\x00': 2, b'I\x00\x00\x003\x00\x00\x00I\x00\x00\x004\x00\x00\x00': 0, b'I\x00\x00\x003\x00\x00\x00I\x00\x00\x005\x00\x00\x00': 1, b'I\x00\x00\x004\x00\x00\x00I\x00\x00\x005\x00\x00\x00': 0}


In [217]:
np.array(basicItem, dtype=object)

array(['I1', 'I2', 'I3', 'I4', 'I5'], dtype=object)

In [218]:
type(item2Set[0])

numpy.ndarray

In [219]:
type(basicItem[0])

str

In [116]:
from collections import defaultdict
newDict = defaultdict(list)
suppCountDict = defaultdict(list)
for item in item2Set:
    suppCountDict[item.tobytes()]=(ap.calculateMinSuppCount(item))
print(suppCountDict[item2Set[0].tobytes()])
print(ap.calculateMinSuppCount(item2Set[1]))

4
4


In [117]:
print(item)

['I4' 'I5']


In [118]:
def getMinSupportItemset(basicItemSet):
    suppCountDict = defaultdict(list)
    for basicItem in basicItemSet:
        suppCountDict[basicItem.tobytes()] = ap.calculateMinSuppCount(basicItem)
    return suppCountDict

In [119]:
newdict = getMinSupportItemSet(item2Set)

In [120]:
print(newdict)

{b'I\x00\x00\x001\x00\x00\x00I\x00\x00\x002\x00\x00\x00': 4, b'I\x00\x00\x001\x00\x00\x00I\x00\x00\x003\x00\x00\x00': 4, b'I\x00\x00\x001\x00\x00\x00I\x00\x00\x004\x00\x00\x00': 1, b'I\x00\x00\x001\x00\x00\x00I\x00\x00\x005\x00\x00\x00': 2, b'I\x00\x00\x002\x00\x00\x00I\x00\x00\x003\x00\x00\x00': 4, b'I\x00\x00\x002\x00\x00\x00I\x00\x00\x004\x00\x00\x00': 2, b'I\x00\x00\x002\x00\x00\x00I\x00\x00\x005\x00\x00\x00': 2, b'I\x00\x00\x003\x00\x00\x00I\x00\x00\x004\x00\x00\x00': 0, b'I\x00\x00\x003\x00\x00\x00I\x00\x00\x005\x00\x00\x00': 1, b'I\x00\x00\x004\x00\x00\x00I\x00\x00\x005\x00\x00\x00': 0}


In [237]:
def getMinSupportItemset(self,basicItemSet):
    suppCountDict = {}
    for basicItem in basicItemSet:
        suppCountDict[basicItem.keys()] = self.calculateMinSuppCount(basicItem)
    return suppCountDict

def printItemsetCount(itemset,suppCountDict):
    minSuppCount = 2
    k = item2Set.shape[1]
    print ('{:=^40}'.format(f' {k}-ItemSet '))
    for item in itemset:
        print(f'{item}: {suppCountDict[item.tobytes()]}')
    print ('{}'.format(f' Item that sastify min support count ({minSuppCount}) '))
    for item in itemset:
        count = suppCountDict[item.tobytes()]
        if(count >= minSuppCount):
            print(f'{item}: {suppCountDict[item.tobytes()]}')
    print ('{:=^40}'.format(f' End of {k}-ItemSet '))

In [242]:
basicItem[0].keys()

AttributeError: 'str' object has no attribute 'keys'

In [238]:
printItemsetCount(item2Set,newdict)

============== 2-ItemSet ===============


KeyError: b'B\x00\x00\x00e\x00\x00\x00e\x00\x00\x00r\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00B\x00\x00\x00r\x00\x00\x00e\x00\x00\x00a\x00\x00\x00d\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [152]:
print(item2Set.shape[1])

2


In [145]:
len(newdict[item2Set[0].tobytes()])

TypeError: object of type 'int' has no len()

In [15]:
print(basicItem)

['I1' 'I2' 'I3' 'I4' 'I5']


In [16]:
def getNextBasicItem(basicItem, k):
    newBasicItem = []
    for n,item in enumerate(basicItem):
        for j in range(n+1,len(basicItem)):
            item1 = item
            item2 = basicItem[j]
            itemSet = np.union1d(item1,item2)
            if(len(itemSet) == k):
                newBasicItem.append(itemSet)
        uniqueBasicItem = np.unique(np.array(newBasicItem),axis=0)
        uniqueBasicItem.astype(object)
    return uniqueBasicItem

In [17]:
newBasicItem = []
newBasicItem = getNextBasicItem(basicItem,2)
new2BasicItem = getNextBasicItem(newBasicItem,3)
new3BasicItem = getNextBasicItem(new2BasicItem,4)
print(len(new3BasicItem))
for item in new3BasicItem:
    print(f'{item},{len(item)},{type(item)}')

5
['I1' 'I2' 'I3' 'I4'],4,<class 'numpy.ndarray'>
['I1' 'I2' 'I3' 'I5'],4,<class 'numpy.ndarray'>
['I1' 'I2' 'I4' 'I5'],4,<class 'numpy.ndarray'>
['I1' 'I3' 'I4' 'I5'],4,<class 'numpy.ndarray'>
['I2' 'I3' 'I4' 'I5'],4,<class 'numpy.ndarray'>


In [18]:
ap.rawSet

array([['I1', 'I2', 'I5', 'None'],
       ['I2', 'I4', 'None', 'None'],
       ['I2', 'I3', 'None', 'None'],
       ['I1', 'I2', 'I4', 'None'],
       ['I1', 'I3', 'None', 'None'],
       ['I2', 'I3', 'None', 'None'],
       ['I1', 'I3', 'None', 'None'],
       ['I1', 'I2', 'I3', 'I5'],
       ['I1', 'I2', 'I3', 'None']], dtype='<U4')

In [19]:
newBasicItem[0]

array(['I1', 'I2'], dtype='<U2')

In [ ]:
# from collections import defaultdict
# newDict = defaultdict(list)
# for item in newBasicItem:
#     itemIndexes = np.where(ap.rawSet == item)
#     counter = 1
#     for i in range(len(itemIndexes[0])):  
#         x = itemIndexes[0][i]
#         y = itemIndexes[1][i]
#         newDict[item].append([x,y])
# for item in newDict:
#     print(f'{item}: {newDict[item]}')

In [ ]:
# item = new2BasicItem[0]
# print(item)
# print(item.dtype)
# output = repr(ap.rawSet).count(item)
# print(itemIndexes)

In [ ]:
# print(np.argwhere(item == ap.rawSet))

In [271]:
#https://stackoverflow.com/questions/56419519/selecting-rows-of-numpy-array-that-contains-all-the-given-values
item = basicItem[0]
print(len(item))
print(item)
mask = np.isin(ap.rawSet,item)
print(mask.sum(axis=1))
vec_mask = np.isin(mask.sum(axis=1), [len(item)])
ids = np.where(vec_mask)
existence = ap.rawSet[ids]
print(mask)
print(vec_mask)
print(ids)
print(existence)
print(len(ids[0]))

4
Beer
[0 1 1 1 0 0 1 1 1 1]
[[False False False False]
 [False False  True False]
 [False False  True False]
 [False False False  True]
 [False False False False]
 [False False False False]
 [False False  True False]
 [False False  True False]
 [False False False  True]
 [False False  True False]]
[False False False False False False False False False False]
(array([], dtype=int64),)
[]
0


In [7]:
np.any(np.isin(ap.rawSet, item), axis=0).sum()

NameError: name 'item' is not defined

In [1042]:
# def find_number_of_occurrences(needle, haystack):
#     needle = tuple(needle)
#     return len([straw for straw in haystack if tuple(straw) == needle])

In [ ]:
# find_number_of_occurrences(item == ap.rawSet)

In [1030]:
# import numpy_indexed as npi
# unique_rows, row_count = npi.count(ap.rawSet, axis=0)

In [ ]:
# for i in range(len(unique_rows)):
#     print(unique_rows[i],',',row_count[i])